In [71]:
import numpy as np
import pandas as pd
import random
import sys
import copy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score
import warnings
warnings.filterwarnings('ignore')

df=pd.read_csv('./ver_2_for_imputation_selected.csv')
df

,Temp_No,Age,Sex,Death,Status,Cancer,Dur_Event_day,Dur_Censored_day,Surtime_day,Etiology,...,smoking,alcohol,Cortical_infarct,RoPE_score,RoPE_score_5_scale,Cancer.1,composite_coagulopathy,Plt_diagnosis,Hb_diagnosis,CRP_diagnosis
0,1,55,0,NaN,0,0,NaN,3170.0,3170,7,...,0,1,1,7,4,0,0,158.0,11.9,0.43
1,2,75,0,NaN,0,0,NaN,2451.0,2451,7,...,0,0,0,3,2,0,0,203.0,12.8,0.21
2,3,81,0,NaN,0,1,NaN,551.0,551,7,...,0,0,0,4,2,1,1,223.0,10.8,0.12
3,4,44,1,NaN,1,0,5.0,NaN,5,7,...,0,1,1,8,4,0,0,199.0,15.2,0.14
4,5,71,1,1.0,1,0,603.0,NaN,603,4,...,0,1,0,3,2,0,1,232.0,10.0,5.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,457,41,1,NaN,0,0,NaN,175.0,175,7,...,1,1,0,6,3,0,0,283.0,13.9,0.14
457,458,53,0,NaN,0,0,NaN,107.0,107,7,...,0,0,0,6,3,0,0,328.0,14.5,0.10
458,459,55,1,NaN,0,0,NaN,177.0,177,3,...,0,0,0,5,3,0,0,262.0,14.6,0.24
459,460,31,0,NaN,0,0,NaN,215.0,215,7,...,0,0,0,8,4,0,0,273.0,11.6,0.10


In [72]:
df=df.dropna(subset=['d_dimer_num_diagnosis'],axis=0)
df.drop(['Temp_No','Cancer.1'],axis=1, inplace=True)
df.drop(['Death','Dur_Event_day','Dur_Censored_day'],axis=1, inplace=True)
df['size'].fillna('Nan', inplace=True)
df['Amt_TEE'].fillna('Nan', inplace=True)
df['Beat '].fillna('Nan', inplace=True)
df['Athero'].fillna('Nan', inplace=True)
df['oldest_d_dimer_num'].fillna(df['oldest_d_dimer_num'].mean(), inplace=True)
df['Homocysteine'].fillna(df['Homocysteine'].mean(), inplace=True)
df['HbA1c'].fillna(df['HbA1c'].mean(), inplace=True)
df['Chol'].fillna(df['Chol'].mean(), inplace=True)
df['HDL'].fillna(df['HDL'].mean(), inplace=True)
df['LDL'].fillna(df['LDL'].mean(), inplace=True)
df['TG'].fillna(df['TG'].mean(), inplace=True)
df['CRP_diagnosis'].fillna(df['CRP_diagnosis'].mean(), inplace=True)

from sklearn.preprocessing import LabelEncoder
def labelencoder(df):
  for c in df.columns:
    if df[c].dtype=='object':
      encoder=LabelEncoder()
      df[c]=encoder.fit_transform(list(df[c].values))
  return df

labelencoder(df)
df


,Age,Sex,Status,Cancer,Surtime_day,Etiology,AF_composite,PFO_positive,size,Amt_TEE,...,TG,smoking,alcohol,Cortical_infarct,RoPE_score,RoPE_score_5_scale,composite_coagulopathy,Plt_diagnosis,Hb_diagnosis,CRP_diagnosis
0,55,0,0,0,3170,7,0,1,48,3,...,71.000000,0,1,1,7,4,0,158.0,11.9,0.43
2,81,0,0,1,551,7,0,0,0,0,...,127.777358,0,0,0,4,2,1,223.0,10.8,0.12
4,71,1,1,0,603,4,0,0,0,0,...,127.777358,0,1,0,3,2,1,232.0,10.0,5.68
5,61,0,1,0,181,7,0,0,0,0,...,82.000000,1,1,1,5,3,0,316.0,14.8,0.10
6,49,1,0,1,955,4,0,1,48,2,...,127.777358,0,1,0,7,4,1,169.0,15.0,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,41,1,0,0,175,7,0,1,10,3,...,103.000000,1,1,0,6,3,0,283.0,13.9,0.14
457,53,0,0,0,107,7,0,0,0,0,...,83.000000,0,0,0,6,3,0,328.0,14.5,0.10
458,55,1,0,0,177,3,0,1,20,3,...,127.777358,0,0,0,5,3,0,262.0,14.6,0.24
459,31,0,0,0,215,7,0,1,4,2,...,82.000000,0,0,0,8,4,0,273.0,11.6,0.10


In [73]:
df.values

array([[5.50e+01, 0.00e+00, 0.00e+00, ..., 1.58e+02, 1.19e+01, 4.30e-01],
       [8.10e+01, 0.00e+00, 0.00e+00, ..., 2.23e+02, 1.08e+01, 1.20e-01],
       [7.10e+01, 1.00e+00, 1.00e+00, ..., 2.32e+02, 1.00e+01, 5.68e+00],
       ...,
       [5.50e+01, 1.00e+00, 0.00e+00, ..., 2.62e+02, 1.46e+01, 2.40e-01],
       [3.10e+01, 0.00e+00, 0.00e+00, ..., 2.73e+02, 1.16e+01, 1.00e-01],
       [3.60e+01, 1.00e+00, 0.00e+00, ..., 3.75e+02, 1.58e+01, 2.40e-01]])

In [74]:
df=pd.DataFrame(df.values, columns=df.columns)
df

,Age,Sex,Status,Cancer,Surtime_day,Etiology,AF_composite,PFO_positive,size,Amt_TEE,...,TG,smoking,alcohol,Cortical_infarct,RoPE_score,RoPE_score_5_scale,composite_coagulopathy,Plt_diagnosis,Hb_diagnosis,CRP_diagnosis
0,55.0,0.0,0.0,0.0,3170.0,7.0,0.0,1.0,48.0,3.0,...,71.000000,0.0,1.0,1.0,7.0,4.0,0.0,158.0,11.9,0.43
1,81.0,0.0,0.0,1.0,551.0,7.0,0.0,0.0,0.0,0.0,...,127.777358,0.0,0.0,0.0,4.0,2.0,1.0,223.0,10.8,0.12
2,71.0,1.0,1.0,0.0,603.0,4.0,0.0,0.0,0.0,0.0,...,127.777358,0.0,1.0,0.0,3.0,2.0,1.0,232.0,10.0,5.68
3,61.0,0.0,1.0,0.0,181.0,7.0,0.0,0.0,0.0,0.0,...,82.000000,1.0,1.0,1.0,5.0,3.0,0.0,316.0,14.8,0.10
4,49.0,1.0,0.0,1.0,955.0,4.0,0.0,1.0,48.0,2.0,...,127.777358,0.0,1.0,0.0,7.0,4.0,1.0,169.0,15.0,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,41.0,1.0,0.0,0.0,175.0,7.0,0.0,1.0,10.0,3.0,...,103.000000,1.0,1.0,0.0,6.0,3.0,0.0,283.0,13.9,0.14
341,53.0,0.0,0.0,0.0,107.0,7.0,0.0,0.0,0.0,0.0,...,83.000000,0.0,0.0,0.0,6.0,3.0,0.0,328.0,14.5,0.10
342,55.0,1.0,0.0,0.0,177.0,3.0,0.0,1.0,20.0,3.0,...,127.777358,0.0,0.0,0.0,5.0,3.0,0.0,262.0,14.6,0.24
343,31.0,0.0,0.0,0.0,215.0,7.0,0.0,1.0,4.0,2.0,...,82.000000,0.0,0.0,0.0,8.0,4.0,0.0,273.0,11.6,0.10


In [75]:
feature=df.drop(['Status','Surtime_day'], axis=1)
label=df[['Status']]
feature

,Age,Sex,Cancer,Etiology,AF_composite,PFO_positive,size,Amt_TEE,High_risk_PFO,Beat,...,TG,smoking,alcohol,Cortical_infarct,RoPE_score,RoPE_score_5_scale,composite_coagulopathy,Plt_diagnosis,Hb_diagnosis,CRP_diagnosis
0,55.0,0.0,0.0,7.0,0.0,1.0,48.0,3.0,0.0,4.0,...,71.000000,0.0,1.0,1.0,7.0,4.0,0.0,158.0,11.9,0.43
1,81.0,0.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,127.777358,0.0,0.0,0.0,4.0,2.0,1.0,223.0,10.8,0.12
2,71.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,127.777358,0.0,1.0,0.0,3.0,2.0,1.0,232.0,10.0,5.68
3,61.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,82.000000,1.0,1.0,1.0,5.0,3.0,0.0,316.0,14.8,0.10
4,49.0,1.0,1.0,4.0,0.0,1.0,48.0,2.0,0.0,2.0,...,127.777358,0.0,1.0,0.0,7.0,4.0,1.0,169.0,15.0,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,41.0,1.0,0.0,7.0,0.0,1.0,10.0,3.0,1.0,1.0,...,103.000000,1.0,1.0,0.0,6.0,3.0,0.0,283.0,13.9,0.14
341,53.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,83.000000,0.0,0.0,0.0,6.0,3.0,0.0,328.0,14.5,0.10
342,55.0,1.0,0.0,3.0,0.0,1.0,20.0,3.0,1.0,1.0,...,127.777358,0.0,0.0,0.0,5.0,3.0,0.0,262.0,14.6,0.24
343,31.0,0.0,0.0,7.0,0.0,1.0,4.0,2.0,0.0,4.0,...,82.000000,0.0,0.0,0.0,8.0,4.0,0.0,273.0,11.6,0.10


In [76]:
label

,Status
0,0.0
1,0.0
2,1.0
3,1.0
4,0.0
...,...
340,0.0
341,0.0
342,0.0
343,0.0


In [85]:
def generate_parent(data):
    gene=[]
    for i in range(200):
        a=random.randint(0,len(data)-1)
        while a in gene:
            a=random.randint(0,len(data)-1)
        gene.append(a)
    return gene

def make_feature_label(feature,label,gene):
    feature_gene=feature.iloc[gene]
    label_gene=label.iloc[gene]
    return feature_gene,label_gene

def get_Fitness(feature,label):
    X_train,X_test,y_train,y_test=train_test_split(feature,label,random_state=50, test_size=0.2)
    rfc=RandomForestClassifier(random_state=50, max_depth=5, n_estimators=25)
    rfc.fit(X_train,y_train)
    pred=rfc.predict(X_test)
    recall=recall_score(pred,y_test)
    return recall

def mutate(gene, feature):
    childGene=copy.deepcopy(gene)
    feature_copy=list(np.arange(len(feature)-1))

    for index in range(0,len(childGene)-1):
        while childGene[index] in feature_copy:
            feature_copy.remove(gene[index])

    for i in range(15):
        index=random.randint(0,len(childGene)-1)
        childGene[index]=random.choice(feature_copy)
    return childGene

In [129]:
n=1
best_parent=generate_parent(feature)
feature_gene, label_gene=make_feature_label(feature,label,best_parent)
first_recall=get_Fitness(feature_gene,label_gene)
print('====={0}st Generation:'.format(n),first_recall,'=====\n')

for i in range(2s0000):
    n+=1
    childGene=mutate(best_parent, feature)
    feature_gene,label_gene=make_feature_label(feature,label,childGene)
    child_recall=get_Fitness(feature_gene,label_gene)
    print('====={0}th Generation:'.format(n),child_recall,'=====')
    if child_recall>=0.5 and child_recall<0.9:
        print('SUCCESS~! The best Gene is %s' %(childGene))
        break
    else:
        continue
    best_parent=childGene

=====1st Generation: 0.0 =====

=====2th Generation: 0.0 =====
=====3th Generation: 1.0 =====
=====4th Generation: 0.0 =====
=====5th Generation: 0.0 =====
=====6th Generation: 1.0 =====
=====7th Generation: 0.0 =====
=====8th Generation: 0.0 =====
=====9th Generation: 1.0 =====
=====10th Generation: 0.0 =====
=====11th Generation: 0.0 =====
=====12th Generation: 0.0 =====
=====13th Generation: 1.0 =====
=====14th Generation: 0.0 =====
=====15th Generation: 0.0 =====
=====16th Generation: 0.0 =====
=====17th Generation: 0.0 =====
=====18th Generation: 0.0 =====
=====19th Generation: 0.0 =====
=====20th Generation: 0.0 =====
=====21th Generation: 0.0 =====
=====22th Generation: 0.0 =====
=====23th Generation: 0.0 =====
=====24th Generation: 0.0 =====
=====25th Generation: 0.0 =====
=====26th Generation: 0.0 =====
=====27th Generation: 0.0 =====
=====28th Generation: 0.0 =====
=====29th Generation: 0.0 =====
=====30th Generation: 0.0 =====
=====31th Generation: 0.0 =====
=====32th Genera

In [130]:
childGene

[271,
 160,
 177,
 322,
 169,
 311,
 156,
 298,
 327,
 138,
 120,
 59,
 283,
 124,
 12,
 309,
 125,
 132,
 191,
 245,
 123,
 29,
 342,
 244,
 87,
 62,
 267,
 239,
 154,
 92,
 293,
 19,
 58,
 289,
 3,
 241,
 215,
 222,
 82,
 13,
 266,
 173,
 140,
 208,
 93,
 317,
 178,
 152,
 131,
 77,
 278,
 48,
 69,
 32,
 166,
 135,
 136,
 234,
 308,
 25,
 50,
 195,
 318,
 328,
 167,
 22,
 186,
 253,
 270,
 205,
 295,
 304,
 111,
 199,
 198,
 214,
 286,
 46,
 34,
 187,
 43,
 331,
 188,
 68,
 196,
 72,
 231,
 247,
 153,
 101,
 248,
 273,
 235,
 182,
 24,
 324,
 224,
 296,
 310,
 100,
 144,
 38,
 110,
 20,
 264,
 249,
 168,
 141,
 171,
 202,
 219,
 329,
 343,
 219,
 147,
 51,
 204,
 312,
 105,
 323,
 194,
 83,
 9,
 88,
 302,
 11,
 44,
 30,
 95,
 221,
 23,
 225,
 162,
 288,
 56,
 299,
 103,
 232,
 18,
 66,
 84,
 129,
 190,
 143,
 85,
 134,
 281,
 73,
 176,
 192,
 207,
 33,
 102,
 142,
 174,
 112,
 40,
 337,
 250,
 116,
 339,
 55,
 157,
 17,
 104,
 183,
 89,
 276,
 237,
 86,
 57,
 90,
 340,
 14,
 7,
 65,


In [132]:
#bestGene=[150, 131, 210, 172, 338, 142, 241, 263, 122, 101, 167, 56, 341, 289, 57, 
#           286, 296, 313, 40, 229, 232, 178, 5, 322, 185, 138, 227, 127, 284, 170, 
#           135, 80, 36, 77, 243, 143, 318, 212, 264, 195, 69, 331, 175, 261, 3, 251, 
#           325, 44, 287, 280, 136, 91, 17, 285, 61, 219, 166, 140, 64, 290, 55, 94,
#           203, 303, 95, 252, 192, 60, 327, 295, 132, 191, 272, 110, 104, 179, 256, 
#           49, 124, 90, 174, 330, 156, 273, 111, 134, 26, 279, 225, 30, 332, 235, 308, 
#           126, 242, 20, 278, 266, 23, 248, 190, 301, 85, 141, 269, 129, 201, 71, 306, 
#           218, 236, 321, 31, 6, 267, 73, 4, 81, 277, 157, 237, 228, 276, 337, 164, 24,
#           226, 253, 270, 45, 214, 299, 302, 158, 103, 115, 50, 163, 271, 33, 62, 70, 292, 
#           149, 168, 59, 314, 189, 21, 204, 27, 187, 79, 245, 162, 184, 249, 53, 344, 169, 
#           298, 270, 46, 128, 84, 67, 205, 221, 297, 19, 97, 1, 153, 160, 239, 82, 22, 9, 186, 37]
#the best Gene
from sklearn.metrics import classification_report
feat, lab= feature.iloc[childGene], label.iloc[childGene]
X_train,X_test,y_train,y_test=train_test_split(feat,lab,test_size=0.2, random_state=50)
rfc=RandomForestClassifier(random_state=50, max_depth=5, n_estimators=100)
rfc.fit(X_train,y_train)
pred=rfc.predict(X_test)
report=classification_report(y_test,pred)
recall=recall_score(y_test,pred)
print(recall)
print(report)

0.0
              precision    recall  f1-score   support

         0.0       0.81      1.00      0.89        29
         1.0       0.00      0.00      0.00         7

    accuracy                           0.81        36
   macro avg       0.40      0.50      0.45        36
weighted avg       0.65      0.81      0.72        36

